In [1]:
# IMPORTS
import numpy as np
import sys
sys.path.insert(0, '../../')
from scripts.model1_scripts.embeddings import TitleEmbeddingModel

/home/evan/ece5424/proj/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ---------------  EXAMPLE DATA ----------------
# db_file = "/content/drive/My Drive/Colab Notebooks/Title2Playlist/title_dataset-PREVIEW.db"
# sql_conn = sqlite3.Connection(db_file)
# cursor = sql_conn.cursor()

# index = 0
# lim = 3
# data = cursor.execute("SELECT title FROM titles LIMIT ? OFFSET ?", (lim, index))

import json

with open ("../../data/playlist/data/mpd.slice.0-999.json") as slice:
    slice_json = json.load(slice)

titles = [playlist['name'] for playlist in slice_json['playlists']]
print(titles)

['Throwbacks', 'Awesome Playlist', 'korean ', 'mat', '90s', 'Wedding', 'I Put A Spell On You', '2017', 'BOP', 'old country ', 'abby ', 'VIBE', 'relax', 'sleep', "90's ", 'New Songs', 'slow hands', "Mom's playlist", 'SARAH', 'melancholy', 'mixtape', 'Sad Songs', "fall '17", '✔️', 'Twenty one pilots', 'run it', 'Winter 2014', 'smooth ', 'Yeet', 'groovy', 'Garage Rock', 'Running 2.0', 'acoustic', 'morning', 'good good', 'Good Vibes', ' indie rock', 'Hits', 'tb', 'Country', '2020', 'Belters', "Road Trippin'", 'Country', 'taylor swift', 'angst', 'Running 2', 'Spring 2016', 'Country', 'Gospel', 'SB', 'Kevin', 'Christmas', 'NB', 'Smooth Jazz', 'Tennessee ', 'funky', 'spring 16', 'Gates', 'Favorite Songs', 'Ski', 'Country', 'hype', 'Awesome Mix', 'Elizabeth', 'PlayStation', 'Ashley', 'chill', 'xx', 'art class', 'Random Tunes', 'boat', 'spanish jams', 'new', 'CR', 'Run!!', "winter '17", 'Workout', 'Traps ', 'Dope', 'Chill', 'rap', 'Gospel', 'Florida', 'Workout2', 'Gym', 'Zoned', 'relax', 'KPOP'

In [3]:
model = TitleEmbeddingModel()


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
import time
then = time.time()
output = model(titles, 30)
print(f"Inference took {time.time() - then}")
print(output.size())
print(output[0][:3])
print(model(titles[0:1], 30)[0][:3])

output2 = model(titles, 30, batch_size=512)
print(output2[0][:3])

print(output.shape)
print(output2.shape)

# print(np.allclose(output, output2, atol=0.001))

Inference took 0.12139153480529785
torch.Size([1000, 768])
tensor([-0.7541,  0.5005,  1.0000], device='cuda:0')
tensor([-0.7541,  0.5005,  1.0000], device='cuda:0')
tensor([-0.7541,  0.5005,  1.0000], device='cuda:0')
torch.Size([1000, 768])
torch.Size([1000, 768])


In [ ]:
output

tensor([[-0.7541,  0.5005,  1.0000,  ...,  1.0000, -0.4139,  0.9913],
        [-0.8613,  0.6134,  1.0000,  ...,  1.0000,  0.4971,  0.9917],
        [-0.6801,  0.4489,  0.9999,  ...,  1.0000, -0.4354,  0.9901],
        ...,
        [-0.8041,  0.4940,  0.9999,  ...,  1.0000, -0.5715,  0.9830],
        [-0.7457,  0.5538,  0.9999,  ...,  1.0000, -0.3959,  0.9787],
        [-0.7510,  0.4595,  0.9999,  ...,  1.0000, -0.7276,  0.9911]])